## This is the capstone project

### Importing libraries

In [7]:
import pandas as pd 
import numpy as np 
import requests 
from bs4 import BeautifulSoup

### Reading the webpage from wikipedia and extracting the table

In [282]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'})

### Getting columns names

In [283]:
# Columns names
colsTemp = My_table.find_all('th')
columns = []
for col in colsTemp:
    columns.append(col.text.replace("\n",""))
columns

['Postcode', 'Borough', 'Neighbourhood']

### Getting data (without the columns) and creating the Data Frame

In [284]:
# get the data
tr = My_table.find_all("tr")
del tr[0]
data = []
for row in tr:
    cells = row.find_all("td")
    items = []
    for cell in cells:
        items.append(cell.text.replace("\n",""))
    data.append(items)
df = pd.DataFrame(data)
df.columns = columns
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Dropping all the Boroughs that are 'Not assigned'

In [285]:
# Drop the 'Not assigned'
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)

### Grouping the dataframe by Postcode, joining the neighbourhoods names and creating a new data frame with the new information

In [286]:
postcodes = df.groupby(['Postcode','Borough'])
data = []
for postcode, dataTemp in postcodes:
    data.append([postcode[0], postcode[1], dataTemp['Neighbourhood'].str.cat(sep=', ')])
df = pd.DataFrame(data)
df.columns = columns
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Asigning the name of the Borough to the Neighbourhoods that are 'Not assigned'

In [290]:
no_neig = df[df['Neighbourhood'] == 'Not assigned'].index
for index in no_neig:
    df.iloc[index]['Neighbourhood'] = df.iloc[index]['Borough']

### getting the shape

In [292]:
df.shape

(103, 3)

## The end